In [12]:
# trd
import MetaTrader5 as mt

# processing
import pandas as pd
from datetime import datetime

---

In [7]:
mt.initialize()

True

In [8]:
login = 0
password = '' 
server = ''

mt.login(login, password, server)

True

In [9]:
mt.account_info()

AccountInfo(login=152702909, trade_mode=0, leverage=2, limit_orders=1024, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=507.39, credit=0.0, profit=0.0, equity=507.39, margin=0.0, margin_free=507.39, margin_level=0.0, margin_so_call=30.0, margin_so_so=0.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='DEMO', server='Exness-MT5Trial', currency='USD', company='Exness Technologies Ltd')

In [18]:
df = pd.DataFrame(mt.copy_rates_range('BTCUSD', mt.TIMEFRAME_D1, datetime(2023, 1, 1), datetime.now()))
df.head()

,time,open,high,low,close,tick_volume,spread,real_volume
0,1672531200,16532.88,16619.48,16492.71,16603.49,79688,427,0
1,1672617600,16602.97,16773.48,16535.88,16665.66,82223,747,0
2,1672704000,16665.36,16773.61,16593.15,16662.49,82161,575,0
3,1672790400,16661.05,16988.17,16639.02,16837.79,83272,371,0
4,1672876800,16839.31,16868.59,16750.72,16814.80,82185,517,0


In [19]:
df.time = pd.to_datetime(df.time, unit='s', utc=True)
df.head()

,time,open,high,low,close,tick_volume,spread,real_volume
0,2023-01-01 00:00:00+00:00,16532.88,16619.48,16492.71,16603.49,79688,427,0
1,2023-01-02 00:00:00+00:00,16602.97,16773.48,16535.88,16665.66,82223,747,0
2,2023-01-03 00:00:00+00:00,16665.36,16773.61,16593.15,16662.49,82161,575,0
3,2023-01-04 00:00:00+00:00,16661.05,16988.17,16639.02,16837.79,83272,371,0
4,2023-01-05 00:00:00+00:00,16839.31,16868.59,16750.72,16814.80,82185,517,0


In [20]:
df.tail()

,time,open,high,low,close,tick_volume,spread,real_volume
404,2024-02-09 00:00:00+00:00,45277.94,48215.97,45214.60,47099.48,170224,1625,0
405,2024-02-10 00:00:00+00:00,47107.30,48167.92,46838.71,47738.99,116633,1627,0
406,2024-02-11 00:00:00+00:00,47730.75,48569.38,47532.30,48284.12,127281,1625,0
407,2024-02-12 00:00:00+00:00,48303.93,50264.19,47653.73,49907.30,164739,1625,0
408,2024-02-13 00:00:00+00:00,49894.69,50323.71,48386.90,48784.10,104074,1627,0
